<a href="https://colab.research.google.com/github/Radperia/SIGNATE_student_2020/blob/master/sklearn_stacking_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>